In [ ]:
import wikipedia as wiki
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.corpora.dictionary import Dictionary
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import itertools
import re
import pandas as pd
from gensim.models.tfidfmodel import TfidfModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Analyzing one Wikipedia article

In [ ]:
page = wiki.page(wiki.random(1))

In [ ]:
print(page.title)

In [ ]:
print(page.url)

In [ ]:
print(page.images)

In [ ]:
print(page.links)

In [ ]:
print(page.summary)

In [ ]:
print(page.content)

In [ ]:
tokens = word_tokenize(page.content)

In [ ]:
lower_tokens = [t.lower() for t in tokens]

In [ ]:
# Creating an initial bag of words.
bag = Counter(lower_tokens)

In [ ]:
print(bag)

### Clean up and preprocessing to remove non-alphabetic characters, stop words, and to lemmatize.

In [ ]:
# Removing non-alphabetic characters.
alpha_only = [t for t in lower_tokens if t.isalpha()]

In [ ]:
# Removing English stop words.
english_stops = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn', '']
no_stops = [t for t in alpha_only if t not in english_stops]

In [ ]:
# Lemmatizing.
word_lemm = WordNetLemmatizer()
lemm = [word_lemm.lemmatize(t) for t in no_stops]

In [ ]:
# Creating an updated and improved bag of words.
newbag = Counter(lemm)
print(newbag.most_common(10))

You can see the most common words in this particular article and glean key points to aid in topic identification.

# Analyzing multiple articles

## Gensim

In [ ]:
multiple_pages = wiki.random(10)

In [ ]:
titles = []
for t in multiple_pages:
    try:
        titles.append(wiki.page(t).title)
    except wiki.DisambiguationError as e:
        titles.append(wiki.page(e.options[0]).title) 
        continue

In [ ]:
print(len(titles))

In [ ]:
contentofpages= []
for c in multiple_pages:
    try:
        contentofpages.append(wiki.page(c).content)
    except wiki.DisambiguationError as e:
        contentofpages.append(wiki.page(e.options[0]).content) 
        continue

In [ ]:
print(len(contentofpages))

In [ ]:
clean_articles = []

for article in contentofpages:
    tokens = word_tokenize(article)
    lower_tokens = [t.lower() for t in tokens]
    alpha_only = [t for t in lower_tokens if t.isalpha()]
    no_stops = [t for t in alpha_only if t not in english_stops]
    lemm = [word_lemm.lemmatize(t) for t in no_stops]
    clean_articles.append(lemm)

In [ ]:
dictionary = Dictionary(clean_articles)

In [ ]:
# Creating a gensim corpus.
corpus = [dictionary.doc2bow(article) for article in clean_articles]

### Gensim Bag-of-Words

In [ ]:
total_word_count = defaultdict(int)
for word_id, word_count in itertools.chain.from_iterable(corpus):
    total_word_count[word_id] += word_count

In [ ]:
sorted_word_count = sorted(total_word_count.items(), key=lambda w: w[1], reverse=True) 

In [ ]:
for word_id, word_count in sorted_word_count[:5]:
    print(dictionary.get(word_id), word_count)

### TF-IDF

In [ ]:
tfidf = TfidfModel(corpus)

In [ ]:
# As an example, we'll use the 3rd article to calculate the significant terms.
tfidf_weights = tfidf[corpus[2]]
sorted_tfidf_weights = sorted(tfidf_weights, key=lambda w: w[1], reverse=True)

In [ ]:
# The top 5 weighted words for that particular article.
for term_id, weight in sorted_tfidf_weights[:5]:
    print(dictionary.get(term_id), weight)

**You can see that both the bag of words looks across the entire dataset to find the most common words. The TF-IDF method, on the other hand, allows us to focus on one particular article to determine how the terms in it are represented across the rest of the dataset.

**Depending on our interests and use cases, you can see how both methods could be valuable.

### Clustering (k-means)

In [ ]:
def tokenize_and_stem(text):
    for article in contentofpages:
        tokens = word_tokenize(article)
        lower_tokens = [t.lower() for t in tokens]
        alpha_only = [t for t in lower_tokens if t.isalpha()]
        no_stops = [t for t in alpha_only if t not in english_stops]
        stems = [word_lemm.lemmatize(t) for t in no_stops]
        return stems

def tokenize_only(text):
    for article in contentofpages:
        tokens = word_tokenize(article)
        lower_tokens = [t.lower() for t in tokens]
        alpha_only = [t for t in lower_tokens if t.isalpha()]
        filtered_tokens = [t for t in alpha_only if t not in english_stops]
        return filtered_tokens

In [ ]:
totalvocab_stemmed =[]
totalvocab_tokenized = []

for i in contentofpages:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [ ]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('There are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame.')

# Alex, this seems to be my problem area. Everything seems to cluster together..only forming 1 cluster.

In [ ]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(contentofpages) #fit the vectorizer to the content

print(tfidf_matrix.shape)

In [ ]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [ ]:
km = KMeans(n_clusters=5)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [ ]:
cluster_articles = {'title': titles, 'text': contentofpages, 'cluster': clusters}

frame = pd.DataFrame(cluster_articles, index = [clusters] , columns = ['title', 'text', 'cluster'])

In [ ]:
frame['cluster'].value_counts() #number of articles per cluster (clusters from 0 to 4)